In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pysrt
import re
import tqdm
import pandas as pd
import logging
from tqdm import tqdm

2023-03-07 17:26:03.975442: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 17:26:03.975466: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [19]:
def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - removes any single character tokens
    Parameters
    ----------
        message : The text message to be preprocessed
    Returns
    -------
        text: The preprocessed text
    """ 
    # Lowercase the twit message
    text = message.lower()
    
    return text
        
# Process for all messages
df = pd.read_csv(r"age.csv",encoding='latin1')
preprocessed = []
for i in range(0,9499):
    message = df['Content'].iloc[i]
    processed_Text = preprocess(message)
    df.loc[i, 'Content'] = processed_Text
    preprocessed.append(str(processed_Text))



In [6]:
df.head(5)

,gender,FileName,Content
0,1,0000e06e07496624211632e8e264126c.txt,paranoia is settin'in sumbuddy's gonna get hur...
1,0,000235a2ba2f48231b7d24e1f08d7878.txt,damn these games kicked my ass today huge wate...
2,0,000c4b6e2468f7d528876fd1a6dffd4c.txt,it is better to conquer yourself than to win t...
3,0,001187432d2a247562082cd0000dec40.txt,its hot over here lol aloha everyone heading o...
4,1,001494c3b74f124a2e3435fff17f376b.txt,he's the lord of all the earth the maker of al...


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Content'],df['age'],test_size=0.1)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [22]:
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.linear_model import RidgeClassifier

# text_preprocessing = Pipeline([('Vect', CountVectorizer())])

# preprocess = ColumnTransformer([('text_preprocessing', text_preprocessing, 'Content')])


# pipeline = Pipeline([
#                 ('preprocess', preprocess),
#                 ('rcf', RidgeClassifier())
#             ])

# df_features = df[['favorite_color','Cat']]
# df_target = df['target']


# pipeline.fit(df_features, df_target)


In [23]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfTransformer

# logreg = Pipeline([('vect', TfidfVectorizer()),
#                 ('tfidf', TfidfTransformer()),
#                 ('clf', LogisticRegression(solver='saga',multi_class='multinomial', C=1e5)),
#                ])
# logreg.fit(X_train, y_train)

# y_pred = logreg.predict(X_test)

# print('accuracy %s' % accuracy_score(y_pred, y_test))

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn import svm
svc = Pipeline([('vect', CountVectorizer()),
                ('scaler',StandardScaler(with_mean=False)),
                ('clf', svm.SVC()),
               ])
svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.5778947368421052


In [8]:
import pickle
pickle.dump(logreg, open('model_age_multilog1.pkl', 'wb'))

In [9]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    return labeled
X_train, X_test, y_train, y_test = train_test_split(df['Content'],df['age'], random_state=0, test_size=0.1)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test


In [10]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.035)
model_dbow.build_vocab([x for x in tqdm(all_data)])

for epoch in range(400):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

print("Loaded the models")

100%|██████████| 9500/9500 [00:00<00:00, 4300225.34it/s]


Loaded the models


In [21]:
import numpy as np

def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')


/tmp/ipykernel_2488429/1976018868.py:15: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  vectors[i] = model.docvecs[prefix]


In [22]:
y_train.shape

(8550,)

In [35]:
logreg = LogisticRegression(n_jobs=1, C=1e5, solver='saga',multi_class='multinomial')
logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)
print('accuracy %s' % accuracy_score(y_pred, y_test))

NameError: name 'train_vectors_dbow' is not defined

In [25]:
import pickle
pickle.dump(logreg, open('model_age_logreg1.pkl', 'wb'))

In [26]:
pickled_model = pickle.load(open('model_age_logreg1.pkl', 'rb'))


In [34]:
import os
list_contents = []
list_ids = []
os.chdir('/data/public-test-data/text')

for filename in os.listdir(os.getcwd()):
    list_ids.append(filename[:-4])
    with open(os.path.join(os.getcwd(), filename), 'r',encoding='latin-1') as f: # open in readonly mode
        list_contents.append(f.read())
        
df = pd.DataFrame(list(zip(list_ids, list_contents)),
               columns =['ID', 'Content'])

print("LOADED THE DF")
############################################
#Text preprocessing
def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - removes any single character tokens
    Parameters
    ----------
        message : The text message to be preprocessed
    Returns
    -------
        text: The preprocessed text
    """ 
    # Lowercase the twit message
    text = message.lower()
    # Replace URLs with a space in the message
    text = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', text)
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub('\$[a-zA-Z0-9]*', ' ', text)
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub('\@[a-zA-Z0-9]*', ' ', text)
    # Replace everything not a letter or apostrophe with a space
    text = re.sub('[^a-zA-Z\']', ' ', text)
    # Remove single letter words
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    
    return text

df = df.reset_index()

for index, row in df.iterrows():
    message = df['Content'].iloc[index]
    processed_Text = preprocess(message)
    df.loc[index, 'Content'] = processed_Text

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    return labeled
X_test_gender = label_sentences(df['Content'], 'Test')
X_test_age = label_sentences(df['Content'], 'Test')
all_data_gender = X_test_gender
all_data_age = X_test_age
############################################
from tqdm import tqdm
#MODEL 
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data_age)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data_age)]), total_examples=len(all_data_age), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

model_dbow1 = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow1.build_vocab([x for x in tqdm(all_data_age)])

for epoch in range(60):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data_age)]), total_examples=len(all_data_age), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha
print("Loaded the models")
############################################

def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        try:
            prefix = vectors_type + '_' + str(i)
            vectors[i] = model.docvecs[prefix]
        except KeyError:
            continue
    return vectors

test_vectors_dbow = get_vectors(model_dbow, len(X_test_gender), 300, 'Test')
test_vectors_dbow1 = get_vectors(model_dbow1, len(X_test_gender), 300, 'Test')


LOADED THE DF


100%|██████████| 334/334 [00:00<00:00, 2858974.56it/s]


Loaded the models


/tmp/ipykernel_2488429/4055670134.py:103: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  vectors[i] = model.docvecs[prefix]


In [28]:
pickle.dump(model_dbow, open('doc2vec.pkl', 'wb'))

In [29]:
pickled_d2v = pickle.load(open('doc2vec.pkl', 'rb'))

In [36]:
test_vectors_dbow = get_vectors(model_dbow, len(X_test_age), 300, 'Test')

/tmp/ipykernel_2488429/4055670134.py:103: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  vectors[i] = model.docvecs[prefix]


In [37]:
pickled_model.predict(test_vectors_dbow[11].reshape(1, -1))

array([0])

In [38]:
y_pred = pickled_model.predict(test_vectors_dbow)
print('accuracy %s' % accuracy_score(y_pred, y_test))

ValueError: Found input variables with inconsistent numbers of samples: [334, 950]